In [1]:
!pip install -U sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 3.2 MB/s eta 0:00:00
  Attempting uninstall: sentence-transformers
    Found existing installation: sentence-transformers 3.2.1
    Uninstalling sentence-transformers-3.2.1:
      Successfully uninstalled sentence-transformers-3.2.1


In [2]:
import torch
import numpy as np
import pandas as pd

from sentence_transformers import SentenceTransformer

from sklearn.decomposition import PCA
from sklearn.metrics import DistanceMetric

import matplotlib.pyplot as plt
import matplotlib as mpl

In [3]:
model = SentenceTransformer("dunzhang/stella_en_1.5B_v5", trust_remote_code=True).cuda()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/397 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/174k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/51.0 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/370 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/289 [00:00<?, ?B/s]

2_Dense_1024/config.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/6.30M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/6.30M [00:00<?, ?B/s]

In [21]:
class Article:
    def __init__(self, id, title, author, text, text_emb, list_prop, list_prop_emb):
        self.id = id
        self.title = title
        self.author = author
        self.text = text
        self.text_emb = text_emb
        self.list_prop_emb = list_prop_emb
        self.list_prop = list_prop

    def __repr__(self):
        return f"Article(id={self.id}, title={self.title}, author={self.author})"

    def get_summary(self, num_sentences=3):
        """Return a short summary of the text (first `num_sentences` sentences)."""
        sentences = self.text.split(".")
        return ". ".join(sentences[:num_sentences]) + "."

In [22]:
# Load the CSV file with articles
import torch.nn.functional as F
data = pd.read_csv('test.csv')

# Initialize the list to store the articles
articole_list = []

# Define a function to compute embeddings
def compute_embeddings(texts, instruction_prefix=""):
    return model.encode(texts, instruction=instruction_prefix)

# Iterate over the dataset to process articles
for i, articol in data.iterrows():
    print(f"Processing Article {i}...")

    # Skip if the article text is NaN or empty
    if pd.isna(articol['text']):
        continue

    # Extract sentences from the article
    sentence_list = [sentence.strip() for sentence in articol['text'].split(".") if sentence]

    # Compute embedding for the entire article text
    text_emb = compute_embeddings([articol['text']])  # Single article as a list

    # Compute embeddings for each sentence
    emb_list = compute_embeddings(sentence_list)

    # Create an article dictionary to store relevant information

    articol_final = Article(
        id=articol['id'],
        title=articol['title'],
        author=articol['author'],
        text=articol['text'],
        text_emb=F.normalize(torch.tensor(text_emb), p=2, dim=1).tolist(),  # Normalize
        list_prop=sentence_list,
        list_prop_emb=F.normalize(torch.tensor(emb_list), p=2, dim=1).tolist()  # Normalize
    )
    articole_list.append(articol_final)


Processing Article 0...
Processing Article 1...
Processing Article 2...
Processing Article 3...
Processing Article 4...
Processing Article 5...
Processing Article 6...
Processing Article 7...
Processing Article 8...
Processing Article 9...
Processing Article 10...
Processing Article 11...
Processing Article 12...
Processing Article 13...
Processing Article 14...
Processing Article 15...
Processing Article 16...
Processing Article 17...
Processing Article 18...
Processing Article 19...
Processing Article 20...
Processing Article 21...
Processing Article 22...
Processing Article 23...
Processing Article 24...
Processing Article 25...
Processing Article 26...
Processing Article 27...
Processing Article 28...
Processing Article 29...
Processing Article 30...
Processing Article 31...
Processing Article 32...
Processing Article 33...
Processing Article 34...
Processing Article 35...
Processing Article 36...
Processing Article 37...
Processing Article 38...
Processing Article 39...
Processing

In [32]:
from sentence_transformers import util

query = "a person is a very very old hero"

query_embedding = model.encode(query)

# Dicționar pentru a stoca cele mai similare propoziții pentru fiecare articol
bestSentancePerArticle = {}

for articol in articole_list:
    # Calculăm similaritatea între fiecare propoziție și query
    cos_simProp = util.cos_sim(articol.list_prop_emb, query_embedding)
    # similarities = model.similarity(query_embedding, articol.list_prop_emb)


    # Obținem scorurile de similaritate și le sortăm în ordine descrescătoare
    sorted_similarities = sorted(enumerate(cos_simProp[0]), key=lambda x: x[1], reverse=True)

    # Salvăm cea mai similară propoziție pentru fiecare articol (index + scor)
    bestSentancePerArticle[articol.id] = sorted_similarities[0]

# Sortăm dicționarul `bestSentancePerArticle` în funcție de scorurile de similaritate, descrescător
sorted_best_sentences = sorted(bestSentancePerArticle.items(), key=lambda x: x[1][1].item(), reverse=True)

# Afișăm primele 5 cele mai similare propoziții pentru query
print("Top-1 most similar sentences to the query:")
for i, (article_id, (index, score)) in enumerate(sorted_best_sentences[:3]):
    articol = next(art for art in articole_list if art.id == article_id)
    print(f"Similarity score: {score.item():.4f}")
    print(f"Article title: {articol.title}")
    print(f"Most similar sentence: {articol.list_prop[index]}")
    print("-" * 40)

Top-1 most similar sentences to the query:
Similarity score: 0.5876
Article title: Trump is USA's antique hero. Clinton will be next president
Most similar sentence: Trump is USA's antique hero
----------------------------------------
Similarity score: 0.5408
Article title: Ann Coulter On C-Span Quotes VDARE.com’s Brimelow–A Racist Is Someone Winning An Argument With A Liberal.
Most similar sentence: 
----------------------------------------
Similarity score: 0.5123
Article title: Reese’s Peanut Butter Cups – Cheap and Full of Toxic Chemicals
Most similar sentence: by ANYA V 
In 1928, a man named H
----------------------------------------
